# ĐỒ ÁN CUỐI KỲ CỦA NHÓM 13:
# DỰ ĐOÁN TẦM GIÁ ĐIỆN THOẠI Ở VIỆT NAM

In [1]:
# Thư viện cần thiết
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
import csv
import math
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram')
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 

# 2. Khám phá dữ liệu + Tiền xử lý dữ liệu 


In [2]:
data_df = pd.read_csv('Phone_Information.csv')

In [3]:
data_df.head()

,Phone_Name,Brand,Screen,OS,Main_Camera,Front_Camera,CPU,RAM,Storage,Battery,Price
0,oppo-reno5,oppo,6.43,Android 11,Chính 64,44,Snapdragon,8.0,128.0,4310,8.690.000
1,oppo-a92,oppo,6.50,Android 10,Chính 48,16,Snapdragon,8.0,128.0,5000,6.490.000₫ -4%
2,iphone-12-mini,iphone,5.40,iOS 14,2 camera 12,12,Apple,4.0,64.0,2227,21.990.000
3,xiaomi-mi-10t-pro,xiaomi,6.67,Android 10,Chính 108,20,Snapdragon,8.0,256.0,5000,12.990.000₫ -5%
4,samsung-galaxy-a12-6gb,samsung,6.50,Android 10,Chính 48,8,MediaTek,6.0,128.0,5000,4.690.000₫ -2%


### Dữ liệu có bao nhiêu dòng và cột?

In [4]:
data_df.shape

(175, 11)

**Quan sát dữ liệu: ta thấy cột Phone_Name không có nhiều ý nghĩa cho việc Huấn luyện mô hình => xoá cột Phone_Name**

In [5]:
df = data_df.drop('Phone_Name',axis =1)


### Các cột có kiểu dữ liệu gì?

In [6]:
df.dtypes

Brand            object
Screen          float64
OS               object
Main_Camera      object
Front_Camera     object
CPU              object
RAM             float64
Storage         float64
Battery           int64
Price            object
dtype: object

**Tại đây ta thấy chúng ta nên chuyển các cột Main_Camera, Front_Camera, Price về dạng số sẽ hợp lý hơn**


### Dữ liệu có các dòng bị lặp không?

In [7]:
df.index.duplicated().sum()

0

### Dữ liệu có bị thiếu không ?

In [8]:
df.isna().sum()

Brand            0
Screen           0
OS              38
Main_Camera     20
Front_Camera    40
CPU             38
RAM             41
Storage         37
Battery          0
Price            1
dtype: int64

Từ đây, chúng ta biết được rằng cần xử lý các cột có giá trị NAN ở phía sau.

# Tiền xử lý dữ liệu ban đầu:

### 1. Cột Price (dự định sẽ là cột output):  loại bỏ ký tự khác số, chuyển về dạng int, đồng thời loại bỏ dòng missing price

In [9]:
df.drop(df[df['Price'].isna()].index,inplace =True)
df['Price']= df['Price'].str.extract(r'(\d+\.\d+[.]?[\d]*)', expand=False)
df['Price']= df['Price'].str.replace(r'[.]','',regex=True).astype(dtype = 'int64',errors = 'ignore')


### 2. Xử lý các cột: Main_Camera, Front_Camera như sau:
#### _ Lấy chuỗi có dạng số float or int
#### _Chuyển về dạng float

In [10]:
df.Main_Camera = pd.to_numeric(df.Main_Camera.str.extract(r'([\d.]+)', expand=False), errors='coerce')

In [11]:
df.Front_Camera = pd.to_numeric(df.Front_Camera.str.extract(r'([\d.]+)', expand=False), errors='coerce')

### Nếu dùng cột Price làm output thì có ổn không?

In [12]:
df['Price'].value_counts()

23990000    6
6990000     6
3990000     5
4990000     5
12990000    4
           ..
890000      1
2290000     1
18990000    1
1250000     1
36000000    1
Name: Price, Length: 99, dtype: int64

Ta thấy rằng cột Price hiện có đến 99 giá trị khác nhau => khó để phân lớp, ta làm mịn dữ liệu cột price bằng cách chia bin thành 5 khoảng và gán nhãn class từ 1->5

In [13]:
df.to_csv('Phone_2021-01-15_dienmayxanh.csv',index = False)

## Tiền xử lý (làm mịn dữ liệu)

Chúng ta sẽ chia bin giá của điện thoại theo luật sau:
- Phân khúc điện thoại **cơ bản** (label = 1): **Price <= 1,000,000**
- Phân khúc điện thoại **phổ thông** (label = 2):  **1,000,000 < Price <=6,000,000**
- Phân khúc điện thoại **trung cấp** (label = 3): **6,000,000 < Price <= 10,000,000**
- Phân khúc điện thoại **cận cao cấp** (label = 4): **10,000,000 < Price <= 18,000,000**
- Phân khúc điện thoại **cao cấp** (label = 5): **Price > 18,000,000**

In [14]:
df.Price.min()

160000

In [15]:
df.Price.max()

50000000

In [16]:
#Chia thành 5 khoảng, min_Price = 160000 ta lấy biên trái là 150000, max_Price =50000000 làm biên phải
cut_bins = [150000,1000000,6000000,10000000,18000000,50000000]

In [17]:
cut_labels = [1,2,3,4,5]

In [18]:
df['Class'] = pd.cut(df['Price'], bins=cut_bins, labels = cut_labels).astype(dtype = 'int')

df.sample(10)


,Brand,Screen,OS,Main_Camera,Front_Camera,CPU,RAM,Storage,Battery,Price,Class
130,vsmart,6.00,Android 9,8.00,5.0,MediaTek,2.0,16.0,3000,1590000,2
57,vivo,6.44,Android 10,48.00,32.0,Snapdragon,8.0,128.0,4500,7690000,3
99,xiaomi,6.30,Android 9,48.00,13.0,Snapdragon,3.0,32.0,4000,3990000,2
127,xiaomi,6.53,Android 10,13.00,5.0,MediaTek,2.0,32.0,5000,1990000,2
118,vsmart,6.50,Android 9,13.00,8.0,Snapdragon,3.0,32.0,5000,2690000,2
156,nokia,1.77,NaN,NaN,NaN,NaN,NaN,NaN,800,370000,1
153,masstel,2.40,NaN,0.08,NaN,NaN,NaN,NaN,2500,400000,1
41,iphone,4.70,iOS 14,12.00,7.0,Apple,3.0,128.0,1821,12990000,4
140,itel,2.40,KaiOS,0.30,NaN,Spreadtrum,NaN,4.0,1900,790000,1
43,iphone,4.70,iOS 14,12.00,7.0,Apple,3.0,64.0,1821,11990000,4


In [19]:
df.to_csv('Phone_2021-01-15_dienmayxanh.csv',index = False)

Đến đây ta thấy cột Price không còn nhiều ý nghĩa cho việc dự đoán => xoá cột Price

In [20]:
df.drop('Price',axis = 1,inplace =True)

## Sau bước đầu khám phá dữ liệu và tiền xử lý thô ta đã có thể đưa ra câu hỏi cần trả lời: "Output được tính từ Input theo công thức nào?
**_Input: Thương hiệu và các cột tính năng của điện thoại bao gồm: Brand, Screen, OS, Main_Camera, Front_Camera, CPU     RAM,Storage, Battery**

**_Output: cột Class**

=> Chúng ta sẽ dự đoán phân khúc tầm giá của một sản phẩm điện thoại mới thông qua thương hiệu và tính năng của nó. Chúng ta thấy rằng, khi ra mắt một sản phẩm smart_phone mới, các nhà sản xuất thường đưa ra tính năng dự kiến trước r mới chính thức đưa ra giá phù hợp với từng thị trường.

# Khám phá dữ liệu (để biết cách tách các tập)

In [21]:
# Cột output hiện có kiểu dữ liệu gì?
df['Class'].dtype

dtype('int32')

In [22]:
# Cột output còn có giá trị thiếu không?
df['Class'].isna().sum()

0

In [23]:
# Tỉ lệ các lớp trong cột output?
df['Class'].value_counts(normalize=True) * 100

2    36.206897
1    23.563218
3    16.666667
5    14.367816
4     9.195402
Name: Class, dtype: float64

- Cột Price có kiểu dữ liệu Objects Chúng ta cần chuyển sang dạng số mới có thể tách được tập
- Cột Price nếu có giá trị thiếu nên ta cần loại bỏ dòng thiếu đi.
- Chúng ta cần làm mịn dữ liệu cột price

# Tiền xử lý (tách các tập)


In [24]:
# Tách X và y
y_pr = df["Class"] 
X_df = df.drop("Class", axis=1)

In [25]:
# Tách tập huấn luyện và tập test theo tỉ lệ 80%:20%
train_X_df, test_X_df, train_y_pr, test_y_pr = train_test_split(X_df, y_pr, test_size=0.2, 
                                                              random_state=0)


In [26]:
train_X_df, val_X_df, train_y_pr, val_y_pr = train_test_split(train_X_df, train_y_pr, test_size=0.2, 
                                                               random_state=0)

In [27]:
train_X_df.shape

(111, 9)

In [28]:
train_y_pr.shape

(111,)

In [29]:
test_X_df.shape

(35, 9)

In [30]:
test_y_pr.shape

(35,)

In [31]:
val_X_df.shape

(28, 9)

In [32]:
val_y_pr.shape

(28,)

# Khám phá dữ liệu (tập huấn luyện)

### 1. Các cột trong tập huấn luyện đang có kiểu dữ liệu gì?

In [33]:
train_X_df.dtypes

Brand            object
Screen          float64
OS               object
Main_Camera     float64
Front_Camera    float64
CPU              object
RAM             float64
Storage         float64
Battery           int64
dtype: object

=> Các cột đang có kiểu dữ liệu phù hợp
Bao gồm 3 cột category : Brand, OS,CPU

## 2. Với cột input dạng số, chúng phân bố như thế nào?

In [34]:
train_X_df.describe()

,Screen,Main_Camera,Front_Camera,RAM,Storage,Battery
count,111.000000,98.000000,87.000000,86.000000,87.000000,111.000000
mean,5.389550,25.544898,13.620690,5.151163,113.333333,3546.927928
std,1.794457,26.003357,9.437975,2.513575,83.788586,1520.485046
min,1.770000,0.080000,2.000000,1.000000,4.000000,800.000000
25%,4.700000,8.000000,8.000000,3.000000,64.000000,2113.500000
50%,6.400000,13.000000,10.000000,4.000000,128.000000,4000.000000
75%,6.505000,48.000000,16.000000,8.000000,128.000000,5000.000000
max,7.590000,108.000000,44.000000,12.000000,512.000000,6000.000000


In [35]:
num_cols = ['Screen', 'Main_Camera', 'Front_Camera', 'RAM','Storage','Battery']
df_train = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df_train.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Screen,Main_Camera,Front_Camera,RAM,Storage,Battery
missing_ratio,0.00,11.70,21.6,22.5,21.6,0.0
min,1.77,0.08,2.0,1.0,4.0,800.0
lower_quartile,4.70,8.00,8.0,3.0,64.0,2113.5
median,6.40,13.00,10.0,4.0,128.0,4000.0
upper_quartile,6.50,48.00,16.0,8.0,128.0,5000.0
max,7.59,108.00,44.0,12.0,512.0,6000.0


Ta nhận thấy, tỉ lệ missing value của các cột dạng số < 30% nên chúng ta có thể xử lý và không cần bỏ cột nào.

### 3. Với các cột input không thể dạng số, các giá trị được phân bố như thế nào?

In [36]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = list(set(train_X_df.columns) - set(num_cols))
df_train = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df_train.agg([missing_ratio, num_values, value_ratios])

,OS,Brand,CPU
missing_ratio,22.5,0,21.6
num_values,9,13,5
value_ratios,"{'Android 10': 62.8, 'Android 9 ': 16.3, 'iOS 14': 12.8, 'Android 8 ': 2.3, 'KaiOS': 1.2, 'EMUI 10 ': 1.2, 'Android 10 ': 1.2, 'iOS 12': 1.2, 'Android 11': 1.2}","{'samsung': 15.3, 'nokia': 12.6, 'iphone': 10.8, 'xiaomi': 9.0, 'oppo': 9.0, 'vivo': 9.0, 'masstel': 8.1, 'realme': 8.1, 'vsmart': 8.1, 'mobell': 3.6, 'energizer': 2.7, 'itel': 2.7, 'huawei': 0.9}","{'Snapdragon': 43.7, 'MediaTek': 28.7, 'Apple': 13.8, 'Exynos': 11.5, 'Spreadtrum': 2.3}"


Các cột không phải số, có tỉ lệ missing value < 30%
Ta nhận thấy, cột Brand, OS có nhiều giá trị khác nhau.

## Tiền xử lý (tập huấn luyện) 

**1.Biến đổi giá trị các cột cho việc huấn luyện hiệu quả hơn:**
- Cột Brand: Lấy top_brand (ví dụ lấy 5 brand phổ biến nhất), những giá trị khác biến thành Others
- Cột OS: trước hết ta điền missing value trong cột với 'No Support'. Tiếp đến,ta nhận thấy có 2 giá trị khác nhau 'Android 10' và 'Android 10 ' do một giá trị bị dư khoảng trắng phía sau số 10, ta tiến hành chuyển 'Android 10 ' thành Android 10. Rồi lấy top_Operations như cột Brand.

**2.Xử lý giá trị thiếu và chuyển cột không phải dạng số về dạng số:**
- Đối với những cột là dạng số:
    - Cột Screen & Battery không có giá trị thiếu.
    - Các cột 'Main_Camera','Front_Camera','RAM','Storage': các cột này bị thiếu chủ yếu do các mẫu điện thoại tương ứng ở phân khúc thấp không hỗ trợ tính năng đó. Vì vậy ta sẽ cho giá trị bằng 0.
- Đối với những cột không phải dạng số:
    - Cột Brand không có giá trị thiếu.
    - Cột CPU : với những giá trị thiếu ta sẽ cho là 'No Support' (Cột OS đã điền giá trị thiếu ở trên bước 1)

##### Bước 1: Biến đổi giá trị các cột cho việc huấn luyện hiệu quả hơn

(class ColTrans tham khảo class ColAdderDropper của BT03-TienXuLy_MoHinhHoa)

In [37]:
class ColTrans_Brand(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_brands =1):
        self.num_top_brands = num_top_brands
        
        
    def fit(self, X_df, y=None):
            X_df_trans =X_df.copy()
            Brand_col = X_df_trans['Brand']
            self.Brand_counts = Brand_col.value_counts()
            Brands = list(self.Brand_counts.index)
            self.top_Brands = Brands[:max(1, min(self.num_top_brands, len(Brands)))]
            return self
    def transform(self, X_df, y=None):
            X_df_trans =X_df.copy()
            Brand = X_df_trans['Brand']
            for br in Brand:
                if br not in self.top_Brands:
                    Brand = Brand.str.replace(br,'Others')
            X_df_trans['Brand'] = Brand
            
            return X_df_trans

In [38]:
# Note: Class này sẽ được gọi sau khi cột Operation đã điền giá trị thiếu
class ColTrans_OS(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_os =1):
        self.num_top_os = num_top_os
    def fit(self, X_df, y=None):
            X_df_trans = X_df.copy()
            OS_col = X_df_trans['OS']
            OS_col = OS_col.fillna('No Support')
            for op in  OS_col:
                if op == 'Android 10 ':
                    OS_col = OS_col.str.replace(op,'Android 10')
            self.OS_counts =  OS_col.value_counts()
            OS_s = list(self.OS_counts.index)
            self.top_OS_s = OS_s[:max(1, min(self.num_top_os, len(OS_s)))]
            return self
    def transform(self, X_df, y=None):
            X_df_trans = X_df.copy()
            OS =  X_df_trans['OS']
            OS = OS.fillna('No Support')
            for op in OS:
                if op not in self.top_OS_s:
                    OS = OS.str.replace(op,'Others')
            X_df_trans['OS'] = OS
            return X_df_trans

#### 2.Xử lý giá trị thiếu:

In [39]:
num_cols = ['Screen','Main_Camera','Front_Camera','RAM','Storage','Battery']
cate_cols = ['Brand','OS','CPU']

num_cols_trans = SimpleImputer(missing_values= np.nan,strategy = 'constant',fill_value = 0)
cate_cols_trans_nan = SimpleImputer(missing_values= np.nan,strategy = 'constant',fill_value = 'No Support')
cate_cols_trans_to_Num = OneHotEncoder(handle_unknown = 'ignore')
cate_cols_trans = make_pipeline(cate_cols_trans_nan,cate_cols_trans_to_Num)

preprocess_pipeline = make_pipeline(ColTrans_Brand(num_top_brands = 6),ColTrans_OS(num_top_os = 4),make_column_transformer((num_cols_trans,num_cols),(cate_cols_trans,cate_cols)),StandardScaler(with_mean = False))
preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)
preprocessed_train_X

array([[3.41477308, 0.46767916, 0.69841857, ..., 0.        , 0.        ,
        0.        ],
       [3.62189866, 0.50665242, 0.79819265, ..., 0.        , 0.        ,
        0.        ],
       [3.64429061, 0.50665242, 0.79819265, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [3.73385843, 4.20911243, 1.99548164, ..., 0.        , 2.10751052,
        0.        ],
       [0.99084399, 0.        , 0.        , ..., 2.42916901, 0.        ,
        0.        ],
       [3.60510469, 1.87071664, 3.19277062, ..., 0.        , 2.10751052,
        0.        ]])

In [40]:
preprocess_pipeline

Pipeline(steps=[('coltrans_brand', ColTrans_Brand(num_top_brands=6)),
                ('coltrans_os', ColTrans_OS(num_top_os=4)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['Screen', 'Main_Camera',
                                                   'Front_Camera', 'RAM',
                                                   'Storage', 'Battery']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='No '
                                                                                            'Support',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Brand', 'OS', 'CPU'])])),
                ('standardscaler', StandardScaler(with_mean=False))])

## Tiền xử lý (tập Validation)

In [41]:
preprocessed_val_X = preprocess_pipeline.transform(val_X_df)

## 3. Tiền xử lý + mô hình hóa

### a. Mô hình Neural Networks

In [42]:
train_y_pr

38     4
90     2
93     2
119    2
139    1
      ..
156    1
34     4
3      4
149    1
50     3
Name: Class, Length: 111, dtype: int32

In [43]:
# tạo full_pipeline
num_cols = ['Screen','Main_Camera','Front_Camera','RAM','Storage','Battery']
cate_cols = ['Brand','OS','CPU']

Neural_Net = MLPClassifier(hidden_layer_sizes= (20), activation='tanh',solver='lbfgs',random_state=0, max_iter=3000)
full_pipeline = make_pipeline(preprocess_pipeline,Neural_Net)

# Thử nghiệm chọn các giá trị khác nhau của các siêu tham số và chọn ra giá trị tốt nhất
train_errs = []
val_errs = []
alphas = [0.1, 1, 10, 100, 1000]
num_top_s = [1, 3, 5, 6, 9, 11]


best_val_err = float('inf'); best_alpha = None; best_num_top_s = None
for alpha in alphas:
    for num_top in num_top_s:
        
        full_pipeline.set_params(pipeline__coltrans_brand__num_top_brands = num_top,pipeline__coltrans_os__num_top_os = num_top, mlpclassifier__alpha = alpha)
        train_df = full_pipeline.fit(train_X_df,train_y_pr)
        train_errs.append((1 - train_df.score(train_X_df,train_y_pr))*100)
        train_val = full_pipeline.predict(val_X_df)
        val_errs.append(((train_val != val_y_pr).mean())*100)
best_val_err = min(val_errs)
min_val_errs_index = val_errs.index(best_val_err)
best_alpha = alphas[min_val_errs_index //6]
best_num_top_s = num_top_s[min_val_errs_index%6]


In [44]:
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('coltrans_brand',
                                  ColTrans_Brand(num_top_brands=11)),
                                 ('coltrans_os', ColTrans_OS(num_top_os=11)),
                                 ('columntransformer',
                                  ColumnTransformer(transformers=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'),
                                                                   ['Screen',
                                                                    'Main_Camera',
                                                                    'Front_Camera',
                                                                    'RAM',
                                                                    'Storage',
                                                                    'Battery']),
                                                                  ('pipeline',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer(fill_value='No '
                                                                                                             'Support',
                                                                                                  strategy='constant')),
                                                                                   ('onehotencoder',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['Brand',
                                                                    'OS',
                                                                    'CPU'])])),
                                 ('standardscaler',
                                  StandardScaler(with_mean=False))])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', alpha=1000,
                               hidden_layer_sizes=20, max_iter=3000,
                               random_state=0, solver='lbfgs'))])

### Độ lỗi tốt nhất trên tập Validation

In [45]:
best_val_err

7.142857142857142

## Huấn luyện mô hình trên X_df, y_pr (Tập train + validation)

In [46]:
full_pipeline.set_params(pipeline__coltrans_brand__num_top_brands = best_num_top_s,pipeline__coltrans_os__num_top_os = best_num_top_s, mlpclassifier__alpha = best_alpha)
final_detailed_model = full_pipeline.fit(X_df,y_pr)


## Đánh giá mô hình Neural Network tìm được

In [47]:
prediction= full_pipeline.predict(test_X_df)

#### Độ lỗi trên tập test

In [48]:
((prediction != test_y_pr).mean())*100

0.0

In [49]:
pd.DataFrame({'Class_test':test_y_pr ,'Class_predict' :prediction})

,Class_test,Class_predict
54,3,3
148,1,1
63,3,3
55,3,3
123,2,2
102,2,2
7,3,3
157,1,1
131,2,2
89,2,2


**=> Mô hình Neural NetWork tốt với độ lỗi trên tập test 0%.**

## b. Mô hình Multinomial Logistic Regression

In [50]:
num_cols = ['Screen','Main_Camera','Front_Camera','RAM','Storage','Battery']
cate_cols = ['Brand','OS','CPU']

LR = LogisticRegression(multi_class='multinomial',max_iter=3000)
full_pipeline_2 = make_pipeline(preprocess_pipeline,LR)

# Thử nghiệm chọn các giá trị khác nhau của các siêu tham số và chọn ra giá trị tốt nhất
train_errs_2 = []
val_errs_2 = []

num_top_s_2 = [1, 3, 5, 6, 9, 11]


best_val_err_2 = float('inf'); best_num_top_s_2 = None

for num_top in num_top_s_2:
        
    full_pipeline.set_params(pipeline__coltrans_brand__num_top_brands = num_top,pipeline__coltrans_os__num_top_os = num_top)
    train_df_2 = full_pipeline_2.fit(train_X_df,train_y_pr)
    train_errs_2.append((1 - train_df_2.score(train_X_df,train_y_pr))*100)
    train_val_2 = full_pipeline_2.predict(val_X_df)
    val_errs_2.append(((train_val_2 != val_y_pr).mean())*100)
best_val_err_2 = min(val_errs_2)
min_val_errs_index_2 = val_errs_2.index(best_val_err_2)
best_num_top_s_2 = num_top_s_2[min_val_errs_index_2]


In [51]:
best_val_err_2

14.285714285714285

## Huấn luyện mô hình trên X_df, y_pr (Tập train + validation)

In [52]:
full_pipeline_2.set_params(pipeline__coltrans_brand__num_top_brands = best_num_top_s,pipeline__coltrans_os__num_top_os = best_num_top_s)
final_detailed_model_2 = full_pipeline_2.fit(X_df,y_pr)

## Đánh giá mô hình Multinomial Logistic Regression tìm được


In [53]:
prediction_2= full_pipeline_2.predict(test_X_df)

#### Độ lỗi trên tập test

In [54]:
((prediction_2 != test_y_pr).mean())*100

2.857142857142857

**=> Mô hình Multinomial Logistic Regression vẫn cho kết quả rất tốt với độ lỗi trên tập test chỉ xấp xỉ 3%**

In [55]:
pd.DataFrame({'Class_test':test_y_pr ,'Class_predict' :prediction_2})

,Class_test,Class_predict
54,3,3
148,1,1
63,3,3
55,3,3
123,2,2
102,2,2
7,3,3
157,1,1
131,2,2
89,2,2


## c. Mô hình Decision Tree

In [56]:
num_cols = ['Screen','Main_Camera','Front_Camera','RAM','Storage','Battery']
cate_cols = ['Brand','OS','CPU']

Decision_Tree = DecisionTreeClassifier()
full_pipeline_3 = make_pipeline(preprocess_pipeline,Decision_Tree)

# Thử nghiệm chọn các giá trị khác nhau của các siêu tham số và chọn ra giá trị tốt nhất
train_errs_3 = []
val_errs_3 = []

num_top_s_3 = [1, 3, 5, 6, 9, 11]


best_val_err_3 = float('inf'); best_num_top_s_3 = None

for num_top in num_top_s_3:
        
    full_pipeline.set_params(pipeline__coltrans_brand__num_top_brands = num_top,pipeline__coltrans_os__num_top_os = num_top)
    train_df_3 = full_pipeline_3.fit(train_X_df,train_y_pr)
    train_errs_3.append((1 - train_df_3.score(train_X_df,train_y_pr))*100)
    train_val_3 = full_pipeline_3.predict(val_X_df)
    val_errs_3.append(((train_val_3 != val_y_pr).mean())*100)
best_val_err_3 = min(val_errs_3)
min_val_errs_index_3 = val_errs_3.index(best_val_err_3)
best_num_top_s_3 = num_top_s_3[min_val_errs_index_3]


In [57]:
best_val_err_3

14.285714285714285

## Huấn luyện mô hình trên X_df, y_pr (Tập train + validation)

In [58]:
full_pipeline_3.set_params(pipeline__coltrans_brand__num_top_brands = best_num_top_s,pipeline__coltrans_os__num_top_os = best_num_top_s)
final_detailed_model_3 = full_pipeline_3.fit(X_df,y_pr)

## Đánh giá mô hình Decision Tree tìm được


In [59]:
prediction_3= full_pipeline_3.predict(test_X_df)

#### Độ lỗi trên tập test

In [60]:
((prediction_3 != test_y_pr).mean())*100

0.0

**=> Mô hìnhDecision Tree vẫn cho kết quả rất tốt với độ lỗi trên tập test chỉ xấp xỉ 0%**

In [61]:
pd.DataFrame({'Class_test':test_y_pr ,'Class_predict' :prediction_3})

,Class_test,Class_predict
54,3,3
148,1,1
63,3,3
55,3,3
123,2,2
102,2,2
7,3,3
157,1,1
131,2,2
89,2,2


## So sánh 3 môn hình: Neural Network, Multinomial Logistic Regression, Decision Tree

### a. Độ lỗi trên tập validation: 
Multinomial Logistic Regression=Decision Tree > Neural Network
### b. Độ lỗi trên tập test:
 Multinomial Logistic Regression > Neural Network = Decision Tree 

**=> trên tập dữ liệu của chúng em thì nhận thấy: Neural Netwwork là mô hình tốt nhất**

# Nhìn lại quá trình

### Khó khăn:
- Thời gian đồ án trong thời gian thi HK1
- Khó khăn trong việc tiền xử lý dữ liệu
- Khó khăn về thu thập dữ phong phú, ít nhiễu, ít thô
- Hạn chế về matplotlib nên không thể trực quan hóa dữ liệu một cách sinh động.
### Những lợi ích mang lại:
- Học được cách tiền xử lý dữ liệu, khám phá dữ liệu, mô hình hóa, đánh giá mô hình hóa
một cách tốt hơn
- Hiểu rõ hơn về quy trình Khoa học dữ liệu .
### Nếu có thêm thời gian:
- Nhóm em sẽ thu thập dữ liệu đa dạng phong phú hơn , thử với
nhiều mô hình khác để độ lỗi trên tập test nhỏ nhất